In [ ]:
import torch
import pyro
import pyro.distributions as dist
import matplotlib
import matplotlib.pyplot as plt
from torch.distributions.constraints import positive

import logging
import os

#import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import scipy

import pyro
import ssms
import lanfactory
#torch.set_default_dtype(torch.float64)
torch.set_default_dtype(torch.float32)

from lanfactory.trainers.torch_mlp import TorchMLP 

import lanfactory
import ssms

In [ ]:
smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.8.1')

pyro.enable_validation(True)
pyro.set_rng_seed(9)
logging.basicConfig(format='%(message)s', level=logging.INFO)

# Set matplotlib settings
%matplotlib inline
plt.style.use('default')

## Define MLP Loader classes

In [ ]:
class CustomTorchMLP(torch.nn.Module):
    def __init__(self, state_dict, network_config):
        super(CustomTorchMLPMod, self).__init__()
        self.weights = []
        self.biases = []
        self.activations = deepcopy(network_config['activations'])
        self.net_depth = len(self.activations)
        self.state_dict = state_dict
        cnt = 0
        for obj in self.state_dict:
            if 'weight' in obj:
                self.weights.append(deepcopy(self.state_dict[obj]).T)
            elif 'bias' in obj:
                self.biases.append(torch.unsqueeze(deepcopy(self.state_dict[obj]), 0))
        #super().__init
        
    def forward(self, input_tensor):
        tmp = input_tensor
        for i in range(0, self.net_depth - 1, 1):
            tmp = torch.tanh(torch.add(torch.matmul(tmp, self.weights[i]), self.biases[i]))
        tmp = torch.add(torch.matmul(tmp, self.weights[self.net_depth - 1]), self.biases[self.net_depth - 1])
        return tmp

## Define model and model config

In [1]:
# Model
from copy import deepcopy
model = "ddm" # for now only DDM (once we have choice probability models --> all models applicable)
model_config = ssms.config.model_config[model].copy() # convenience
network_config = pickle.load(open('nets/d27193a4153011ecb76ca0423f39a3e6_' + \
                                  'ddm_torch__network_config.pickle', 'rb'))


NameError: name 'ssms' is not defined

### Load with TorchMLP from LANfactory package

In [ ]:
# Initialize network class
torch_net = TorchMLP(network_config = network_config,
                     input_shape = len(model_config['params']) + 2,
                     generative_model_id = None)

# Load weights and biases
torch_net.load_state_dict(torch.load('nets/d27193a4153011ecb76ca0423f39a3e6_ddm_torch_state_dict.pt', 
                          map_location=torch.device('cpu')))


### Use custom TorchMLP loader

In [ ]:
# Initialize custom pytorch network
# custom_torch_net = CustomTorchMLP(torch_net.state_dict(), 
#                                   network_config)

custom_torch_net = CustomTorchMLPMod(torch_net.state_dict(), 
                                     network_config)
custom_torch_net.eval()